# Demonstration Timbre Transfer of Ensemble
This notebook demonstrates how an ensemble sounds like if the timbre is transferred from virtual instruments from the SLAKH dataset to timbre that is generated by DDSP based on real world recordings.

In [1]:
# Cut audio
import numpy as np
from dataloader_slakh import SlakhDataset
import os
# import numpy as np
from IPython.display import Audio, display
import matplotlib.pyplot as plt
from pydub import AudioSegment
import soundfile as sf

%load_ext autoreload
%autoreload 2


/home/nicolas/workspace/ma/ma4/ddspzart/.venv/lib/python3.8/site-packages/requests/__init__.py:102: RequestsDependencyWarning: urllib3 (1.26.4) or chardet (5.0.0)/charset_normalizer (2.0.7) doesn't match a supported version!
  warnings.warn("urllib3 ({}) or chardet ({})/charset_normalizer ({}) doesn't match a supported "


In [2]:
# Get Audio Snippet
TARGET = "Trumpet"
train_dataset = SlakhDataset(target=TARGET, seq_duration=5.0) #TODO does not work on my machine

i = 0
print([train_dataset.tracks[i].targets[j].instrument for j in range(
    len(train_dataset.tracks[i].targets))])
# print(train_dataset.tracks[0].targets)
print([train_dataset.tracks[i].targets[j].instrument for j in [3, -6, -1]])

audios = [train_dataset.tracks[i].targets[j].get_total_audio()
          for j in [2, -7, -5, -2]]

# Cut Audio
t = np.linspace(0, 4+0.06/60*100, len(audios[0]))
t_1 = 3 + (.19 / 60*100)
t_2 = 3 + (.4 / 60*100)
t_cut_ = t[t > t_1]
t_cut = t_cut_[t_cut_ < t_2]

audios_cut = []
titles = ["Trumpet", "Trombone", "Guitar", "Flute"]
for i, a in enumerate(audios):
    cut = a[t > t_1]
    cut = np.array(cut[t_cut_ < t_2])
    audios_cut.append(cut)
    sf.write("data/audio/example_"+titles[i]+".wav", cut, 44_100)

display(Audio(np.sum(np.array(audios_cut), axis=0), rate=44100))
for i, audio_ in enumerate(audios_cut):
    display(Audio(audio_, rate=44100))
    plt.figure()
    plt.plot(t_cut, audio_)
    plt.title(titles[i])
    plt.ylabel("Sound amplitude")
    plt.xlabel("t[s]")


FileNotFoundError: [Errno 2] No such file or directory: 'C:/Users/User/Documents/GitHub/uni/ddspzart/workspace_2/source_separation/data/OneExampleRecording/train'

In [ ]:
# Perform Timbre Transfer (here as an example in Google Colab to not require DDSP implementation)
# Then, load the timbre transformed files.
# Trained DDSP Models are not provided (500 MByte per model). We can distribute them if desired.

from audio2numpy import open_audio
sound_flute_flute, sr = open_audio(
    "data/audio/example_flute_new_timbre_flute.wav")
sound_guitar_guitar, sr = open_audio(
    "data/audio/example_guitar_new_timbre_guitar.wav")
sound_trombone_horn, sr = open_audio(
    "data/audio/example_trombone_new_timbre_horn.wav")
sound_trumpet_trumpet, sr = open_audio(
    "data/audio/example_trumpet_new_timbre_trumpet.wav")


In [ ]:
# Combine and play
Audio(sound_flute_flute+sound_guitar_guitar+sound_guitar_guitar +
      sound_trombone_horn+sound_trumpet_trumpet, rate=sr)


As can be heard above, the output is not really natural even though a natural model was assumed.
However, and that is one major learning, the sound of an instrument is not only defined by its timbre. Instead, artifacts arising from artificial oscillations lead to the strange sound that was generated.